In [26]:
!pip install datasets
!pip install transformers
!pip install peft
!pip install evaluate
!pip install -U datasets==2.20.0 pyarrow==15.0.2 transformers==4.44.2 evaluate==0.4.2 --no-cache-dir
!pip install -q datasets evaluate accelerate scikit-learn pandas matplotlib
!pip install -U "transformers>=4.41" accelerate safetensors
!pip install -U bitsandbytes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 175.1 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 380.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.3
    Uninstalling transformers-4.57.3:
      Successfully uninstalled transformers-4.57.3


  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached transformers-4.57.3-py3-none-any.whl (12.0 MB)
Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [27]:
!pip install --upgrade --no-cache-dir git+https://github.com/Shannu3766/bi_influence.git

  Cloning https://github.com/Shannu3766/bi_influence.git to /tmp/pip-req-build-1c1wz8p5
  Running command git clone --filter=blob:none --quiet https://github.com/Shannu3766/bi_influence.git /tmp/pip-req-build-1c1wz8p5
  Resolved https://github.com/Shannu3766/bi_influence.git to commit 981f2a3a9b906864b4f65765cb42335892489a46
  Preparing metadata (setup.py) ... done
  Created wheel for adaptive_lora: filename=adaptive_lora-2.1.0-py3-none-any.whl size=5843 sha256=f2e1b4fe11573ec9b8b069783adcb60e58a472773f041bd1732a93955dba7e82
  Stored in directory: /tmp/pip-ephem-wheel-cache-8_ir6nlp/wheels/f8/5f/fc/97210c723ddb66cb0b5c21449c734301ecc806d8d629ad0829
Successfully built adaptive_lora


In [28]:
import warnings
import os
import logging
os.environ["WANDB_DISABLED"] = "true"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore")
# logging.set_verbosity_error()

In [29]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
from torch.utils.data import DataLoader
from adaptive_lora.callbacks import AdaptiveLoRACallback
import numpy as np
print("torch:", torch.__version__)
print("cuda devices:", torch.cuda.device_count())

torch: 2.9.0+cu126
cuda devices: 1


In [30]:
import os
import warnings
import torch
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    logging
)
from peft import get_peft_model, LoraConfig
import evaluate

In [31]:
# Silence all unnecessary outputs
os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore")
logging.set_verbosity_error()


In [ ]:
dataset = load_dataset("glue", "sst2")

# 2️⃣ Model checkpoint and labels
model_checkpoint = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative": 0, "Positive": 1}

# 3️⃣ Load model
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=2,
    id2label=id2label,
    label2id=label2id,
    device_map="auto"
)

In [33]:
from transformers import AutoTokenizer, DataCollatorWithPadding

# 4️⃣ Load tokenizer and fix pad token
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

# TinyLlama / LLaMA-style tokenizer does NOT need add_prefix_space
# (that's mostly for GPT-2-like models)
# So we don't pass add_prefix_space=True here.

if tokenizer.pad_token is None:
    # For LLaMA-like models, use eos_token as pad_token
    tokenizer.pad_token = tokenizer.eos_token
    model.resize_token_embeddings(len(tokenizer))

model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False

MAX_LEN = 128  # SST-2 sentences are short; 128 is usually plenty


def clean_text(s):
    if s is None:
        return ""
    # strip whitespace + collapse multiple spaces/newlines
    return " ".join(str(s).strip().split())


def tokenize_function(examples):
    # 🧠 Make it more "instructional" for a causal LM
    texts = [
        f"Review: {clean_text(x)}\nSentiment:"
        for x in examples["sentence"]
    ]

    return tokenizer(
        texts,
        truncation=True,
        padding=False,   # ✅ dynamic padding via DataCollator
        max_length=MAX_LEN,
    )


# Remove raw text + idx, keep label
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["sentence", "idx"],
)

# Trainer expects "labels" by default
if "label" in tokenized_dataset["train"].column_names:
    tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

tokenized_dataset.set_format("torch")

# 6️⃣ Data collator handles dynamic padding
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    pad_to_multiple_of=8,  
)


Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [43]:
accuracy = evaluate.load("accuracy")
rank = 8
def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = np.argmax(preds, axis=1)
    return {"accuracy": accuracy.compute(predictions=preds, references=labels)}

# 8️⃣ LoRA config
peft_config = LoraConfig(
    task_type="SEQ_CLS",
    r=rank,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=[   
        "q_proj", "k_proj", "v_proj", "o_proj", 
        "gate_proj", "up_proj", "down_proj"
    ]
)
model = get_peft_model(model, peft_config)


In [44]:
from peft.tuners.lora import LoraLayer

def get_lora_module_names(peft_model):
    names = []
    for name, module in peft_model.named_modules():
        if isinstance(module, LoraLayer):
            names.append(name)
    return names

lora_names = get_lora_module_names(model)   # `model` is your PEFT-wrapped model
print(f"🔢 Number of LoRA modules: {len(lora_names)}\n")

# for n in lora_names:
    # print(" ", n)
num_lora_modules = len(lora_names)
TOTAL_RANK_BUDGET = num_lora_modules*rank  # example

avg_rank_per_module = TOTAL_RANK_BUDGET / num_lora_modules
print(
    f"💡 With TOTAL_RANK_BUDGET={TOTAL_RANK_BUDGET} over "
    f"{num_lora_modules} modules, avg rank ≈ {avg_rank_per_module:.2f}"
)


🔢 Number of LoRA modules: 154

💡 With TOTAL_RANK_BUDGET=1232 over 154 modules, avg rank ≈ 8.00


In [45]:
training_args = TrainingArguments(
    disable_tqdm=False,
    output_dir="./SST2_adaptive_results",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=2,
    logging_steps=500,
    eval_strategy="epoch",
    eval_steps=500,
    bf16=True,              
    report_to="none",
    save_strategy="no",
)


In [46]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [55]:

from torch.utils.data import DataLoader
train_dataset = tokenized_dataset["train"]
eval_dataset = tokenized_dataset["validation"]

val_dataloader = DataLoader(
    eval_dataset,
    batch_size=8,
    shuffle=False,
    collate_fn=data_collator
)

In [56]:
from adaptive_lora.callbacks import AdaptiveLoRACallback

adaptive_callback = AdaptiveLoRACallback(
    val_dataloader=val_dataloader,
    total_rank=TOTAL_RANK_BUDGET,
    tau=0.9,      
    verbose=True
)

In [57]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[adaptive_callback],
)

In [ ]:
trainer.train()


--- AdaptiveLoRA: Preparing ranks for Epoch 1 ---
Computing BI importance scores (pre-training)...


Computing BI scores:   0%|          | 0/109 [00:00<?, ?it/s]

Allocating new ranks based on BI scores...
Applying new ranks to LoRA modules for this epoch...
  - base_model.model.base_model.model.model.layers.0.self_attn.q_proj: r=8 (Unchanged, Score: 0.9897)
  - base_model.model.base_model.model.model.layers.0.self_attn.k_proj: r=8 (Unchanged, Score: 1.0000)
  - base_model.model.base_model.model.model.layers.0.self_attn.v_proj: r=8 (Unchanged, Score: 0.9961)
  - base_model.model.base_model.model.model.layers.0.self_attn.o_proj: r=8 (Unchanged, Score: 0.9950)
  - base_model.model.base_model.model.model.layers.0.mlp.gate_proj: r=8 (Unchanged, Score: 0.9943)
  - base_model.model.base_model.model.model.layers.0.mlp.up_proj: r=8 (Unchanged, Score: 1.0000)
  - base_model.model.base_model.model.model.layers.0.mlp.down_proj: r=8 (Unchanged, Score: 0.9992)
  - base_model.model.base_model.model.model.layers.1.self_attn.q_proj: r=8 (Unchanged, Score: 1.0000)
  - base_model.model.base_model.model.model.layers.1.self_attn.k_proj: r=8 (Unchanged, Score: 0.992

Epoch,Training Loss,Validation Loss


In [51]:
met = trainer.evaluate()
print(met)

{'eval_loss': 0.5248047113418579, 'eval_accuracy': {'accuracy': 0.7}, 'eval_runtime': 1.0239, 'eval_samples_per_second': 9.766, 'eval_steps_per_second': 4.883, 'epoch': 5.0}
